```mermaid
flowchart LR
subgraph video generator
  direction LR
  VS[video source] --> overlay
  overlay --> R[GL Sink]
end
  R-.->GL
subgraph video display
  direction LR
  GL[GL driver] --> DPW[Display Window]
end

```

In [ ]:
!gst-launch-1.0 \
    videotestsrc ! video/x-raw,width=320,height=240 ! \
    timeoverlay halignment=center valignment=center font-desc="Sans, 32" ! \
    autovideosink

```mermaid
flowchart LR
subgraph video generator
  direction LR
  VS[video source] --> overlay
  overlay --> TCPC[tcp client]
end
  TCPC-.->TCPS
subgraph video display
  direction LR
  TCPS[tcp server]-->MJPGS[MJPG splitter]
  MJPGS --> Id[IPython.display]
end

```

In [ ]:
import socket
from IPython.display import display, Image, clear_output
from io import BytesIO

def receive_mjpeg_stream(host, port):
    # Create a socket and bind it to the host and port
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
        server_socket.bind((host, port))
        server_socket.listen(1)  # Listen for incoming connections (1 backlog)
        print(f"Server is listening on {host}:{port}")

        # Accept connections from outside
        (client_socket, address) = server_socket.accept()
        print(f"Connection accepted from {address}")

        # The buffer to store received data
        buffer = bytearray()
        try:
            # Continuously receive data from the client
            while True:
                data = client_socket.recv(4096)  # Adjust buffer size as necessary
                if not data:
                    break  # No more data, terminate the loop
                
                # Append data to the buffer
                buffer.extend(data)
                
                # Process each frame
                while True:
                    start = buffer.find(b'\xff\xd8')
                    end = buffer.find(b'\xff\xd9', start + 2)

                    # Check if we have a valid frame
                    if start != -1 and end != -1:
                        # Extract the frame
                        jpg = buffer[start:end + 2]
                        # Clear the buffer up to the end of the frame
                        buffer = buffer[end + 2:]

                        # Create a BytesIO stream from the JPEG data
                        img_stream = BytesIO(jpg)

                        # Create an Image object and display it
                        img = Image(data=img_stream.read())
                        clear_output(wait=True)
                        display(img)
                    else:
                        break  # No complete frame, exit loop to receive more data
        finally:
            client_socket.close()  # Close the client socket
            print("Connection closed.")

# Specify the host and port to listen on
HOST = '0.0.0.0'  # Listen on all network interfaces
PORT = 12345      # Port number

# Run the server
receive_mjpeg_stream(HOST, PORT)
